In [249]:
import sys
import importlib
from fpdf import FPDF
import pandas as pd
import numpy as np

dir_scr = "../utils/"
sys.path.append(dir_scr)

import mypdfutils
importlib.reload(mypdfutils)


from fpdf import FPDF

class PDF(FPDF):
    def __init__(self, font="Arial", font_size=12):
        super().__init__()

        # Page dimensions (A4 size by default)
        self.w = 210
        self.h = 297
        self.default_font = font
        self.default_font_size = font_size

        # Set the default margins
        self.left_margin = 10
        self.top_margin = 10
        self.right_margin = 10
        self.bottom_margin = 10

        self.gray_color = (128, 128, 128)
        self.black_color = (0, 0, 0)

        self.box_width = 20
        self.min_box_width = 30
        self.box_height = 5
        self.box_spacing = 17

    def set_default_font(self):
        self.set_font(self.default_font, size=self.default_font_size)

    def set_small_font(self):
        self.set_font(self.default_font, size=8)

    def set_small_font_bold(self):
        self.set_font(self.default_font, size=8, style="B")

    def center_text(self, text, font=None, size=None, bold=False, custom_height=None, px=0):
        # Use the default font and size if not provided
        font = font or self.default_font
        size = size or self.default_font_size

        # Set font and size
        if bold:
            self.set_font(font, style="B", size=size)
        else:
            self.set_font(font, size=size)

        # Get the width and height of the text
        text_width = self.get_string_width(text)
        text_height = self.font_size

        # Use custom height if provided, otherwise use the vertical center of the page
        if custom_height is not None:
            y = custom_height - text_height / 2
        else:
            y = (self.h - text_height) / 2

        # Set margins before writing the text
        self.set_left_margin(self.left_margin)
        self.set_top_margin(self.top_margin)
        self.set_right_margin(self.right_margin)
        self.set_auto_page_break(auto=True, margin=self.bottom_margin)

        # Write the text at the calculated position
        self.text((self.w - text_width) / 2 + px, y, text)

    def footer(self):
        return
        # self.set_y(-15)
        # self.set_font('Helvetica', 'I', 8)
        # self.set_text_color(128)
        # self.cell(0, 10, 'Page ' + str(self.page_no()), 0, 0, 'C')

    def header(self):
        pdf.image("/home/nic/al/commissaire/resources/headerDocumenti.png", 2, 7, self.w)

    def add_box(self, x, y, text):
        self.set_fill_color(255, 255, 255)
        self.rect(x, y, self.box_width, self.box_height, "DF")
        self.set_text_color(*self.black_color)
        self.set_xy(x, y)
        self.cell(self.box_width, self.box_height, text, 0, 0, "C")

    def add_gray_text(self, x, y, text):
        self.set_text_color(*self.gray_color)
        self.set_xy(x, y)
        self.cell(self.box_width, self.box_height, text, 0, 0, "C")

    def create_boxes(self, x=20, y=40, data=["Pista", "167754", "07", "Q", "0345"]):
        titles = ["Tipo gara", "N° Gara", "Cod. Reg.", "Cin", "Cod. Soc."]


        for i in range(5):
            self.set_small_font()
            self.add_gray_text(x, y - 5, titles[i])
            self.set_small_font_bold()
            self.add_box(x, y, data[i])
            x += self.box_width + self.box_spacing


# Create PDF
pdf = PDF()
pdf.add_page()

# pdf.center_text("CAMPIONATI ITALIANI GIOVANILI SU PISTA", custom_height=22, size=14, bold=1, px=2)
# pdf.center_text("Velodromo di Forlì (FC), 1-4 Agosto 2023", custom_height=28, size=12, bold=1, px=2)

pdf.center_text("FEDERAZIONE CICLISTICA ITALIANA", custom_height=25, size=15, bold=1, px=2)
pdf.create_boxes(x=20, y=38)

# pdf.center_text("ORDINE D'ARRIVO", custom_height=58, size=15, bold=1)
# pdf.center_text("Ordre d'Arrivée", custom_height=65, size=13, bold=1)

pdf.center_text("ELENCO ISCRITTI - Tutti", custom_height=60, size=15, bold=1)
pdf.center_text("Campionati Italiani Pista 2023 - Giovanili", custom_height=67, size=13, bold=1)
pdf.center_text("Velodromo Glauco Servadei - ASD Consorzio delle Soc. Ciclist. Romagnole", custom_height=74, size=12, bold=0)



dir_data = "../../data/Iscritti_167754.xls"
df = pd.read_excel(dir_data)
team        = df["NomeSocieta"].str.lstrip().unique()
idGara      = df["IdGara"][0]
nomeGara    = df["NomeGara"][0]

df.drop(columns=['IdGara', 'NomeGara', 'Nazionalità', 'CodiceFiscale', 'Sesso', 'Riserva', 'Scadenza Certificato'], inplace=True)
nome = np.array(df["NomeTesserato"])
prov = np.array(df["Provincia"]).astype(str)
cat = np.array(df["Categoria"])

df["NomeTesserato"] = [f"{a:s} ({b:s}) [{c:s}]" for (a,b,c) in zip(nome, prov, cat)]

# df = df[df["Categoria"]=="AL"]

df = df.sort_values(by="DorsaleNumero",ignore_index=True)


################ TABLE
table = df[["DorsaleNumero", "CodiceUci", "CodiceFCI", "NomeTesserato", "NomeSocieta"]]
N     = len(table)
ord   = np.arange(1,N+1)
new_column_names = {
    'DorsaleNumero': 'Dors',
    'CodiceUci': 'UCI Id',
    'CodiceFCI': 'Licenza',
    'NomeTesserato': 'Cognome Nome',
    'NomeSocieta': 'Società',
}
table.rename(columns=new_column_names, inplace=True)
table.insert(0, "Ord", ord)




# Full table
pdf.set_font(pdf.default_font, size=9)
col_width = [10,10,25,17,60,60]
col_al    = ["C","C","C","C","L","L"]
ccol_width = np.cumsum(col_width)
row_height = 6
max_page = 40
max_page0 = 30
pagen = 0

x_table, y_table = 15, 80
x,y=x_table,y_table

pdf.set_draw_color(140, 140, 140)

def render_header(y):
    pdf.set_small_font_bold()
    header_content = list(table.keys())
    for j, header in enumerate(header_content):
        pdf.set_xy(x + ccol_width[j - 1] if j > 0 else x_table, y_table)
        pdf.cell(col_width[j], row_height, header, border=1, ln=0, align="C", fill=True)
    y += row_height
    pdf.set_small_font()
    return y


for i, row in table.iterrows():
    if i == 0:
        x,y = 15, 80
        y = render_header(y)

    elif ((pagen==0) and ((i%max_page0)==0)) or ((pagen>0) and (((i-max_page0)%max_page)==0)):
        # New page or the beginning of a new page, render the header

        pdf.add_page()
        x,y = 15, 35
        x_table, y_table = x,y
        y= render_header(y)
        pagen+=1

    if i % 2 == 0:
        pdf.set_fill_color(240, 245, 245)
    else:
        pdf.set_fill_color(255, 255, 255)

    for j, value in enumerate(row):
        if list(table.keys())[j] == "Dors":
            pdf.set_small_font_bold()
        else:
            pdf.set_small_font()

        pdf.set_xy(x + ccol_width[j - 1] if j > 0 else x_table, y)
        pdf.cell(col_width[j], row_height, str(value), border=1, ln=0, align=col_al[j], fill=True)
    y += row_height


pdf.output("test.pdf", 'F')

/tmp/ipykernel_23235/822696913.py:161: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  table.rename(columns=new_column_names, inplace=True)


''

In [250]:
[12,13,4556]

,DorsaleNumero,NomeTesserato,CodiceFCI,Categoria,CodiceUci,DataNascita,NomeSocieta,CodiceSocieta,Note,Cognome,Nome,Provincia
0,1,ZANNONI SERENA (RE) [DA],A091111,DA,10033988376,03/10/2008,XPC BELTRAMI TSA COOPERATORI,07F0103,Iscrizione CR.,ZANNONI,SERENA,RE
1,1,PIGNATO MARTINA (RA) [ED],A072439,ED,10054916229,25/11/2010,BICIFESTIVAL,07N1350,Iscrizione CR.,PIGNATO,MARTINA,RA
2,1,TASSI EDOARDO (MO) [AL],A015476,AL,10031073528,18/07/2008,V.C. PONTENURE 1957-F. ZEPPI,07D0051,Iscrizione CR.,TASSI,EDOARDO,MO
3,1,BERARDI MICHELE (RN) [ES],A073638,ES,10032974223,05/10/2009,ASD G. C. FAUSTO COPPI,07B0164,Iscrizione CR.,BERARDI,MICHELE,RN
4,2,CASUBOLO GIULIA (RE) [DA],A154905,DA,10081929113,21/07/2008,XPC BELTRAMI TSA COOPERATORI,07F0103,Iscrizione CR.,CASUBOLO,GIULIA,RE
...,...,...,...,...,...,...,...,...,...,...,...,...
253,98,MAGAGNOTTI ALESSIO (TN) [AL],A058753,AL,10032407276,27/01/2007,C.C. FORTI E VELOCI,20L0001,Iscrizione CR. TRENTO,MAGAGNOTTI,ALESSIO,TN
254,99,CARESIA EDOARDO AUGUSTO (TN) [AL],A125059,AL,10059983669,08/03/2007,C.C. FORTI E VELOCI,20L0001,Iscrizione CR. TRENTO,CARESIA,EDOARDO AUGUSTO,TN
255,100,LEONI DANIELE (BZ) [AL],A100050,AL,10034595133,08/02/2008,LIBERTAS RAIFFEISEN LAIVES ASD,21W0011,Iscrizione CR. Bolzano,LEONI,DANIELE,BZ
256,101,GOTTARDI MANUEL (BZ) [AL],A124077,AL,10059334476,03/12/2008,G.S. ALTO ADIGE - SÜDTIROLPOST,21Y0037,Iscrizione CR. Bolzano,GOTTARDI,MANUEL,BZ


In [200]:
df.keys()

Index(['DorsaleNumero', 'NomeTesserato', 'CodiceFCI', 'Categoria', 'CodiceUci',
       'DataNascita', 'NomeSocieta', 'CodiceSocieta', 'Note', 'Cognome',
       'Nome', 'Provincia'],
      dtype='object')

In [164]:
table[0][0]

KeyError: 0

In [212]:
reg_cod = df["Provincia"]

reg_

0      RA
1      RA
2      RA
3      PD
4      PD
       ..
253    RE
254    RE
255    FI
256    LI
257    FI
Name: Provincia, Length: 258, dtype: object

In [216]:
df["Provincia"]

0      RA
1      RA
2      RA
3      PD
4      PD
       ..
253    RE
254    RE
255    FI
256    LI
257    FI
Name: Provincia, Length: 258, dtype: object

['CONTI MARTINA (RA)',
 'DONATI FLAVIO (RA)',
 'SIGNORINI LUCA (RA)',
 'CECCARELLO LORENZO (PD)',
 'QUAGLIO CHRISTIAN (PD)',
 'CAPUZZO MARCO (PD)',
 'ROS FEDERICO (PN)',
 'TABOGA GIOELE (UD)',
 'DALLE CRODE LORENZO (GO)',
 'PITTA JACOPO (UD)',
 'ROVERETTO SIMONE (UD)',
 'MORETTI ANDREA (PN)',
 'CAPORASO CHIARA (SA)',
 'CARRAFIELLO RICCARDO (SA)',
 'MAZZONI MIRCO (TO)',
 'NADALI AURORA (VR)',
 'MELOTTO FILIPPO (VR)',
 'SCAPINI LUCIA (VR)',
 'ROSSIGNOLI MATILDE (VR)',
 'MELOTTO THOMAS (VR)',
 'FERRARI IRENE (MN)',
 'ACUTI MARIA (MN)',
 'ELIOTROPIO FILIPPI SOFIA (TP)',
 'CRUCIATA STEFANO (TP)',
 'BORELLA LORENZO (VE)',
 'FORLIN DANIELE (PD)',
 "NIERI NICCOLO' (PO)",
 'FERRUZZI EMANUELE (FI)',
 'MORONI GABRIELE (FI)',
 'VEGLIA ALBERTO (CN)',
 'SBRAVATI ALICE (VR)',
 "SCAMPERLE NICOLO' (VR)",
 'MORELLO GAIA (VI)',
 'SANTELLA GIULIA (MN)',
 'BORDIGNON SILVIA (VI)',
 'OSPIZIO EDOARDO (VR)',
 'BERARDI MICHELE (RN)',
 'BRAMINI GORETTI GIORGIO (PG)',
 'LO PICCOLO DOMENICO (PA)',
 "GALLI' LUCREZI

ValueError: Unknown format code 's' for object of type 'float'

In [124]:
table

,Ord,Dors,UCI Id,Licenza,Cognome Nome,Società
0,1,5,10031079992,A015685,CONTI MARTINA,A. D.PED. AZZURRO.RINASCITA
1,2,10,10030852044,A004455,DONATI FLAVIO,A. D.PED. AZZURRO.RINASCITA
2,3,11,10065364947,A131741,SIGNORINI LUCA,A. D.PED. AZZURRO.RINASCITA
3,4,25,10059762993,A124674,CECCARELLO LORENZO,A.C.D. MONSELICE
4,5,53,10075622493,A132895,QUAGLIO CHRISTIAN,A.C.D. MONSELICE
...,...,...,...,...,...,...
253,254,2,10053845185,A095324,MANFREDI LEONARDO,XPC BELTRAMI TSA COOPERATORI
254,255,2,10081929113,A154905,CASUBOLO GIULIA,XPC BELTRAMI TSA COOPERATORI
255,256,30,10032604815,A063827,MASI AURORA,ZHIRAF GUERCIOTTI TOSCANO
256,257,46,10031017146,A013010,FRANCESCHINI EMMA,ZHIRAF GUERCIOTTI TOSCANO


KeyError: "None of [Index(['DorsaleNumero', 'CodiceUci', 'CodiceFCI', 'NomeTesserato',\n       'NomeSocieta'],\n      dtype='object')] are in the [columns]"

In [107]:
table

,DorsaleNumero,CodiceUci,CodiceFCI,NomeTesserato,NomeSocieta
0,5,10031079992,A015685,CONTI MARTINA,A. D.PED. AZZURRO.RINASCITA
1,10,10030852044,A004455,DONATI FLAVIO,A. D.PED. AZZURRO.RINASCITA
2,11,10065364947,A131741,SIGNORINI LUCA,A. D.PED. AZZURRO.RINASCITA
3,25,10059762993,A124674,CECCARELLO LORENZO,A.C.D. MONSELICE
4,53,10075622493,A132895,QUAGLIO CHRISTIAN,A.C.D. MONSELICE
...,...,...,...,...,...
253,2,10053845185,A095324,MANFREDI LEONARDO,XPC BELTRAMI TSA COOPERATORI
254,2,10081929113,A154905,CASUBOLO GIULIA,XPC BELTRAMI TSA COOPERATORI
255,30,10032604815,A063827,MASI AURORA,ZHIRAF GUERCIOTTI TOSCANO
256,46,10031017146,A013010,FRANCESCHINI EMMA,ZHIRAF GUERCIOTTI TOSCANO


In [91]:
table

,DorsaleNumero,NomeTesserato,CodiceFCI,Categoria,CodiceUci,DataNascita,NomeSocieta,CodiceSocieta,Note,Cognome,Nome,Provincia
0,5,CONTI MARTINA,A015685,DA,10031079992,12/01/2007,A. D.PED. AZZURRO.RINASCITA,07P0798,Iscrizione CR.,CONTI,MARTINA,RA
1,10,DONATI FLAVIO,A004455,AL,10030852044,11/04/2007,A. D.PED. AZZURRO.RINASCITA,07P0798,Iscrizione CR.,DONATI,FLAVIO,RA
2,11,SIGNORINI LUCA,A131741,AL,10065364947,11/10/2007,A. D.PED. AZZURRO.RINASCITA,07P0798,Iscrizione CR.,SIGNORINI,LUCA,RA
3,25,CECCARELLO LORENZO,A124674,ES,10059762993,29/06/2009,A.C.D. MONSELICE,03X0144,Iscrizione CR.,CECCARELLO,LORENZO,PD
4,53,QUAGLIO CHRISTIAN,A132895,AL,10075622493,09/10/2007,A.C.D. MONSELICE,03X0144,Iscrizione CR.,QUAGLIO,CHRISTIAN,PD
...,...,...,...,...,...,...,...,...,...,...,...,...
253,2,MANFREDI LEONARDO,A095324,ES,10053845185,11/05/2010,XPC BELTRAMI TSA COOPERATORI,07F0103,Iscrizione CR.,MANFREDI,LEONARDO,RE
254,2,CASUBOLO GIULIA,A154905,DA,10081929113,21/07/2008,XPC BELTRAMI TSA COOPERATORI,07F0103,Iscrizione CR.,CASUBOLO,GIULIA,RE
255,30,MASI AURORA,A063827,ED,10032604815,05/12/2009,ZHIRAF GUERCIOTTI TOSCANO,08H3021,Iscrizione CR. TOSCANA,MASI,AURORA,FI
256,46,FRANCESCHINI EMMA,A013010,DA,10031017146,17/01/2007,ZHIRAF GUERCIOTTI TOSCANO,08H3021,Iscrizione CR. TOSCANA,FRANCESCHINI,EMMA,LI


In [70]:
df

,NomeGara,DorsaleNumero,NomeTesserato,CodiceFCI,Categoria,CodiceUci,Nazionalità,DataNascita,NomeSocieta,CodiceSocieta,CodiceFiscale,Sesso,Note,Cognome,Nome,Riserva,Scadenza Certificato,Provincia
0,CAMPIONATI ITALIANI PISTA 2023 - GIOVANILI,5,CONTI MARTINA,A015685,DA,10031079992,ITA,12/01/2007,A. D.PED. AZZURRO.RINASCITA,07P0798,CNTMTN07A52H199X,F,Iscrizione CR.,CONTI,MARTINA,NO,2024-05-14,RA
1,CAMPIONATI ITALIANI PISTA 2023 - GIOVANILI,10,DONATI FLAVIO,A004455,AL,10030852044,ITA,11/04/2007,A. D.PED. AZZURRO.RINASCITA,07P0798,DNTFLV07D11H199N,M,Iscrizione CR.,DONATI,FLAVIO,NO,2023-09-18,RA
2,CAMPIONATI ITALIANI PISTA 2023 - GIOVANILI,11,SIGNORINI LUCA,A131741,AL,10065364947,ITA,11/10/2007,A. D.PED. AZZURRO.RINASCITA,07P0798,SGNLCU07R11H199Y,M,Iscrizione CR.,SIGNORINI,LUCA,NO,2023-10-14,RA
3,CAMPIONATI ITALIANI PISTA 2023 - GIOVANILI,25,CECCARELLO LORENZO,A124674,ES,10059762993,ITA,29/06/2009,A.C.D. MONSELICE,03X0144,CCCLNZ09H29A001P,M,Iscrizione CR.,CECCARELLO,LORENZO,NO,2024-01-19,PD
4,CAMPIONATI ITALIANI PISTA 2023 - GIOVANILI,53,QUAGLIO CHRISTIAN,A132895,AL,10075622493,ITA,09/10/2007,A.C.D. MONSELICE,03X0144,QGLCRS07R09F382C,M,Iscrizione CR.,QUAGLIO,CHRISTIAN,NO,2023-09-21,PD
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
253,CAMPIONATI ITALIANI PISTA 2023 - GIOVANILI,2,MANFREDI LEONARDO,A095324,ES,10053845185,ITA,11/05/2010,XPC BELTRAMI TSA COOPERATORI,07F0103,MNFLRD10E11H223T,M,Iscrizione CR.,MANFREDI,LEONARDO,NO,2023-12-05,RE
254,CAMPIONATI ITALIANI PISTA 2023 - GIOVANILI,2,CASUBOLO GIULIA,A154905,DA,10081929113,ITA,21/07/2008,XPC BELTRAMI TSA COOPERATORI,07F0103,CSBGLI08L61F463J,F,Iscrizione CR.,CASUBOLO,GIULIA,NO,2024-01-26,RE
255,CAMPIONATI ITALIANI PISTA 2023 - GIOVANILI,30,MASI AURORA,A063827,ED,10032604815,ITA,05/12/2009,ZHIRAF GUERCIOTTI TOSCANO,08H3021,MSARRA09T45A564B,F,Iscrizione CR. TOSCANA,MASI,AURORA,NO,2024-01-09,FI
256,CAMPIONATI ITALIANI PISTA 2023 - GIOVANILI,46,FRANCESCHINI EMMA,A013010,DA,10031017146,ITA,17/01/2007,ZHIRAF GUERCIOTTI TOSCANO,08H3021,FRNMME07A57C415K,F,Iscrizione CR. TOSCANA,FRANCESCHINI,EMMA,NO,2024-02-22,LI


In [38]:
table = df.to_html(index=False).replace('table border="1" class="dataframe"', 'table border="1" cellpadding="5" cellspacing="0" style="background-color: #F0F5FF; border-style: ridge; color: #4040C0" width="100%"').replace('<td>', '<td width=100>')
table

'<table border="1" cellpadding="5" cellspacing="0" style="background-color: #F0F5FF; border-style: ridge; color: #4040C0" width="100%">\n  <thead>\n    <tr style="text-align: right;">\n      <th>Nome</th>\n      <th>Punteggio</th>\n      <th>Posizione</th>\n    </tr>\n  </thead>\n  <tbody>\n    <tr>\n      <td width=100>John Doe</td>\n      <td width=100>85</td>\n      <td width=100>1</td>\n    </tr>\n    <tr>\n      <td width=100>Jane Smith</td>\n      <td width=100>92</td>\n      <td width=100>2</td>\n    </tr>\n    <tr>\n      <td width=100>Mike Johnson</td>\n      <td width=100>78</td>\n      <td width=100>3</td>\n    </tr>\n  </tbody>\n</table>'

In [11]:
from pyfpdf import FPDF, HTMLMixin

ModuleNotFoundError: No module named 'pyfpdf'

In [7]:
table.replace('<table border="1"', '<table border="1" width="100%"')

'<table border="1" width="100%" class="dataframe">\n  <thead>\n    <tr style="text-align: right;">\n      <th></th>\n      <th>Nome</th>\n      <th>Punteggio</th>\n      <th>Posizione</th>\n    </tr>\n  </thead>\n  <tbody>\n    <tr>\n      <th>0</th>\n      <td>John Doe</td>\n      <td>85</td>\n      <td>1</td>\n    </tr>\n    <tr>\n      <th>1</th>\n      <td>Jane Smith</td>\n      <td>92</td>\n      <td>2</td>\n    </tr>\n    <tr>\n      <th>2</th>\n      <td>Mike Johnson</td>\n      <td>78</td>\n      <td>3</td>\n    </tr>\n  </tbody>\n</table>'

In [6]:
table

'<table border="1" class="dataframe">\n  <thead>\n    <tr style="text-align: right;">\n      <th></th>\n      <th>Nome</th>\n      <th>Punteggio</th>\n      <th>Posizione</th>\n    </tr>\n  </thead>\n  <tbody>\n    <tr>\n      <th>0</th>\n      <td>John Doe</td>\n      <td>85</td>\n      <td>1</td>\n    </tr>\n    <tr>\n      <th>1</th>\n      <td>Jane Smith</td>\n      <td>92</td>\n      <td>2</td>\n    </tr>\n    <tr>\n      <th>2</th>\n      <td>Mike Johnson</td>\n      <td>78</td>\n      <td>3</td>\n    </tr>\n  </tbody>\n</table>'

In [ ]:
PDF